# Cognee ile Kalıcı Belleğe Sahip AI Ajanları Oluşturma

Bu not defteri, bağlam farkındalığına sahip AI sistemleri oluşturmak için bilgi grafikleri, anlamsal arama ve oturum yönetimini birleştiren açık kaynaklı bir AI belleği olan [**cognee**](https://www.cognee.ai/) kullanarak gelişmiş bellek yeteneklerine sahip zeki AI ajanları oluşturmayı gösterir.

## 🎯 Öğrenme Hedefleri

Bu eğitimin sonunda şunları anlayacaksınız:
- **Gömülü Bilgi Grafikleri Oluşturma**: Yapılandırılmamış metni yapılandırılmış, sorgulanabilir bilgiye dönüştürme
- **Oturum Belleği Uygulama**: Otomatik bağlam koruması ile çoklu dönüşlü konuşmalar oluşturma
- **Konuşmaları Kalıcı Hale Getirme**: Önemli etkileşimleri uzun vadeli bellek için isteğe bağlı olarak saklama
- **Doğal Dil Kullanarak Sorgulama**: Yeni konuşmalarda geçmiş bağlamı erişme ve kullanma
- **Belleği Görselleştirme**: Ajanınızın bilgi grafiğindeki ilişkileri keşfetme


## 🏗️ Ne İnşa Edeceksiniz

Bu eğitimde, kalıcı hafızaya sahip bir **Kodlama Asistanı** oluşturacağız:

### 1. **Bilgi Tabanı Oluşturma**
   - Geliştirici profili ve uzmanlık bilgilerini alır
   - Python programlama prensiplerini ve en iyi uygulamaları işler
   - Geliştiriciler ve yapay zeka asistanları arasındaki geçmiş konuşmaları depolar

### 2. **Oturum Farkındalığına Sahip Konuşmalar**
   - Aynı oturumdaki birden fazla soru arasında bağlamı korur
   - Her soru/cevap çiftini otomatik olarak önbelleğe alır ve verimli bir şekilde geri çağırır
   - Konuşma geçmişine dayalı tutarlı ve bağlamsal yanıtlar sağlar

### 3. **Uzun Süreli Hafıza**
   - Önemli konuşmaları uzun süreli hafızaya kaydeder
   - Yeni etkileşimleri bilgilendirmek için bilgi tabanından ve geçmiş oturumlardan ilgili anıları geri çağırır
   - Zamanla gelişen bir bilgi tabanı oluşturur

### 4. **Akıllı Hafıza Geri Çağırma**
   - Tüm depolanan bilgiler arasında ilgili bilgileri bulmak için grafik farkındalıklı anlamsal arama kullanır
   - Aramaları veri alt gruplarına göre filtreler (geliştirici bilgileri vs. prensipler)
   - Kapsamlı yanıtlar sağlamak için birden fazla veri kaynağını birleştirir


## 📋 Ön Koşullar ve Kurulum

### Sistem Gereksinimleri

Başlamadan önce aşağıdakilere sahip olduğunuzdan emin olun:

1. **Python Ortamı**
   - Python 3.9 veya daha üstü
   - Sanal ortam (önerilir)
   
2. **Redis Cache** (Oturum Yönetimi için Gerekli)
   - Yerel Redis: `docker run -d -p 6379:6379 redis`
   - Ya da yönetilen bir Redis hizmeti kullanabilirsiniz
   
3. **LLM API Erişimi**
   - OpenAI API anahtarı veya diğer sağlayıcılar (bkz. [dokümantasyon](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Veritabanı Yapılandırması**
   - Varsayılan olarak herhangi bir yapılandırma gerekmez. Cognee dosya tabanlı veritabanları (LanceDB ve Kuzu) kullanır.
   - İsteğe bağlı olarak, Azure AI Search'ü bir vektör deposu olarak kurabilirsiniz (bkz. [dokümantasyon](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Ortam Yapılandırması

Proje dizininizde aşağıdaki değişkenlerle bir `.env` dosyası oluşturun:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cognee'nin Bellek Mimarisi Anlama

### Cognee Nasıl Çalışır

Cognee, basit anahtar-değer depolamanın ötesine geçen gelişmiş bir bellek sistemi sunar:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Temel Bileşenler:

1. **Bilgi Grafiği**: Varlıkları, ilişkileri ve anlamsal bağlantıları depolar
2. **Vektör Gömüleri**: Tüm depolanan bilgiler arasında anlamsal arama yapmayı sağlar
3. **Oturum Önbelleği**: Oturumlar sırasında ve arasında konuşma bağlamını korur
4. **DüğümSetleri**: Verileri hedefe yönelik erişim için mantıksal kategorilere ayırır

### Bu Eğitimdeki Bellek Türleri:

- **Kalıcı Bellek**: Bilgi grafiğinde uzun vadeli depolama
- **Oturum Belleği**: Redis önbelleğinde geçici konuşma bağlamı
- **Anlamsal Bellek**: Tüm veriler arasında vektör tabanlı benzerlik araması


## 📦 Gerekli Paketleri Yükleyin

Oturum yönetimi için Redis desteğiyle birlikte Cognee'yi yükleyin:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Ortamı Başlat ve Kütüphaneleri Yükle

Emin olun:
1. Redis çalışıyor (örneğin, Docker ile: `docker run -d -p 6379:6379 redis`)
2. Önbellek modüllerini içe aktarmadan önce ortam değişkenleri ayarlandı
3. Gerekirse, çekirdeği yeniden başlatın ve hücreleri sırayla çalıştırın

Aşağıdaki hücre şunları yapacaktır:
1. Ortam değişkenlerini `.env` dosyasından yükleyecek
2. Cognee'yi LLM ayarlarınızla yapılandıracak
3. Oturum yönetimi için önbelleği etkinleştirecek
4. Tüm bileşenlerin düzgün bir şekilde bağlı olduğunu doğrulayacak


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Depolama Dizinlerini Yapılandırma

Cognee, işlemleri için iki ayrı dizin kullanır:
- **Veri Kök Dizini**: Alınan belgeleri ve işlenmiş verileri depolar
- **Sistem Kök Dizini**: Bilgi grafiği veritabanını ve sistem meta verilerini içerir

Bu eğitim için aşağıdaki gibi izole edilmiş dizinler oluşturacağız:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Bellek Durumunu Sıfırla

Bellek sistemimizi oluşturmaya başlamadan önce, her şeyin sıfırdan başladığından emin olalım.

> 💡 **İpucu**: Bu adımı atlayabilirsiniz, eğer bu not defterini daha sonra kullandığınızda önceki çalışmalardan kalan mevcut anıları korumak istiyorsanız.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Bölüm 1: Bilgi Tabanı Oluşturma

### Geliştirici Asistanı için Veri Kaynakları

Kapsamlı bir bilgi tabanı oluşturmak için üç tür veri alacağız:

1. **Geliştirici Profili**: Kişisel uzmanlık ve teknik geçmiş
2. **Python En İyi Uygulamaları**: Python'un Zen'i ve pratik yönergeler
3. **Tarihi Konuşmalar**: Geliştiriciler ve yapay zeka asistanları arasındaki geçmiş Soru-Cevap oturumları

Bu çeşitli veri, ajanımızın:
- Kullanıcının teknik bağlamını anlamasını
- Önerilerde en iyi uygulamaları kullanmasını
- Önceki başarılı etkileşimlerden öğrenmesini sağlar


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Verileri Bilgi Grafiğine Dönüştür

Şimdi ham metnimizi yapılandırılmış bir hafızaya dönüştüreceğiz. Bu süreç:

1. **Verileri NodeSets'e ekler**: Bilgiyi mantıksal kategorilere organize eder
   - `developer_data`: Geliştirici profili ve konuşmalar
   - `principles_data`: Python en iyi uygulamaları ve yönergeler

2. **Cognify Pipeline'ı çalıştırır**: Varlıkları, ilişkileri çıkarır ve gömme işlemleri yapar
   - Anahtar kavramları belirler
   - İlgili bilgiler arasında anlamsal bağlantılar oluşturur
   - Vektör gömmeleri oluşturur

LLM metni işlerken ve grafik yapısını oluştururken bu işlem birkaç dakika sürebilir:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Bilgi Grafiğini Görselleştir

Bilgi grafiğimizin yapısını inceleyelim. Görselleştirme şunları gösterir:
- **Düğümler**: Metinden çıkarılan varlıklar (kavramlar, teknolojiler, kişiler)
- **Kenarlar**: Varlıklar arasındaki ilişkiler ve bağlantılar
- **Kümeler**: Anlamsal benzerliklerine göre gruplanmış ilgili kavramlar

Grafiği etkileşimli olarak keşfetmek için oluşturulan HTML dosyasını tarayıcınızda açın:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Hafızayı Memify ile Zenginleştirin

`memify()` fonksiyonu bilgi grafiğini analiz eder ve veri hakkında akıllı kurallar oluşturur. Bu süreç:
- Kalıpları ve en iyi uygulamaları belirler
- İçeriğe dayalı uygulanabilir yönergeler oluşturur
- Farklı bilgi alanları arasındaki ilişkileri kurar

Bu kurallar, ajanın soruları yanıtlarken daha bilinçli kararlar almasına yardımcı olur. İkinci bir görselleştirme yakalamak, grafiğin zenginleştirildikten sonra nasıl yoğunlaştığını karşılaştırmanıza olanak tanır.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Bölüm 2: Akıllı Hafıza Erişimi

### Gösterim 1: Belgeler Arası Bilgi Entegrasyonu

Artık bilgi grafiğimiz oluşturulduğuna göre, Cognee'nin karmaşık soruları yanıtlamak için birden fazla kaynaktan nasıl bilgi birleştirdiğini test edelim.

İlk sorgu şunları gösterir:
- **Anlamsal anlayış**: Açıkça belirtilmese bile ilgili kavramları bulma
- **Çapraz referanslama**: Geliştirici profili ile Python prensiplerini birleştirme
- **Bağlamsal akıl yürütme**: En iyi uygulamaları belirli uygulamalara uyarlama

### Gösterim 2: NodeSets ile Filtrelenmiş Arama

İkinci sorgu, bilgi grafiğinin belirli alt kümelerini hedeflemenin nasıl mümkün olduğunu gösterir:
- Sadece `principles_data` içinde arama yapmak için `node_name` parametresini kullanır
- Belirli bir bilgi alanından odaklanmış yanıtlar sağlar
- Alan spesifik bilgilere ihtiyaç duyduğunuzda faydalıdır


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Bölüm 3: Oturum Yönetimi Ayarları

### Konuşma Hafızasını Etkinleştirme

Oturum yönetimi, birden fazla etkileşimde bağlamı korumak için çok önemlidir. Burada şunları yapacağız:

1. **Kullanıcı Bağlamını Başlatma**: Oturum takibi için bir kullanıcı profili oluşturun veya alın
2. **Önbellek Motorunu Yapılandırma**: Konuşma geçmişini depolamak için Redis'e bağlanın
3. **Oturum Değişkenlerini Etkinleştirme**: Sorgular arasında kalıcı olan bağlam değişkenlerini ayarlayın

> ⚠️ **Önemli**: Bunun için Redis'in çalışıyor olması ve ortamınızda `CACHING=true` ayarının yapılmış olması gereklidir


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Yardımcı Fonksiyon: Oturum Geçmişini Görüntüle

Bu yardımcı fonksiyon, Redis'te saklanan konuşma geçmişini incelememizi sağlar. Şu durumlar için faydalıdır:
- Oturum yönetimini hata ayıklama
- Konuşmaların önbelleğe alındığını doğrulama
- Ajanın hangi bağlama sahip olduğunu anlama


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Oturum 1: Async Destek Laboratuvarı — İlk Soru

`async-support-lab` oturumunu, büyük bir web kazıyıcı için telemetri dostu asyncio desenleri hakkında sorular sorarak başlatın. Grafik zaten asyncio, aiohttp ve izleme uygulamaları hakkında bilgi sahibi, bu yüzden yanıt önceki konuşmaları yansıtmalı ve yeni soruya uygun şekilde uyarlanmalıdır.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## İlk Etkileşimden Sonra Oturum 1 Belleğini İnceleme

İlk sorudan hemen sonra `show_history(session_1)` çalıştırmak, Cognee'nin hem istemi hem de tamamlamayı Redis'e yazdığını doğrular. Eşzamanlılık rehberiyle birlikte bir giriş görmelisiniz.


In [ ]:
await show_history(session_1)

## Oturum 1: Veri Modelleri Üzerine Takip

Şimdi şu soruyu soruyoruz: "Dataclasses yerine Pydantic'i ne zaman seçmeliyim?" Aynı oturum kimliğini kullanarak. Cognee, Python prensiplerini ve önceki FastAPI konuşmalarını birleştirerek bağlamın adlandırılmış bir oturum içinde devam ettiğini gösteren ayrıntılı tavsiyeler sunmalı.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Oturum 1 Geçmişinin Her İki Dönüşü İçerdiğini Doğrulayın

Bir başka `show_history(session_1)` çağrısı, iki Soru-Cevap girişini göstermelidir. Bu, Mem0 laboratuvarının "hafıza tekrar oynatma" adımına uyar ve ek dönüşlerin aynı transkripti genişlettiğini kanıtlar.


In [ ]:
await show_history(session_1)

## Oturum 2: Tasarım İnceleme Konusu — Yeni Oturum

Konular arasındaki izolasyonu göstermek için `design-review-session` başlatıyoruz ve olay incelemeleri için günlük tutma rehberliği istiyoruz. Temel bilgi tabanı aynı olsa da, yeni oturum kimliği transkriptleri ayrı tutar.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## İnceleme Oturumu 2 Geçmişi

`show_history(session_2)` yalnızca tasarım inceleme istemi/cevap çiftini listelemelidir. Bunu Oturum 1 ile karşılaştırarak, Cognee'nin paylaşılan bilgi grafiğini yeniden kullanırken bağımsız transkriptleri nasıl koruduğunu vurgulayın.


In [ ]:
await show_history(session_2)

## Özet

Tebrikler! Kodlama asistanınıza Cognee tarafından desteklenen gerçek bir uzun vadeli hafıza katmanı kazandırdınız.

Bu eğitimde, ham geliştirici içeriğini (kod, belgeler, sohbetler) alıp, ajanınızın arama yapabileceği, mantık yürütebileceği ve sürekli geliştirebileceği bir grafik + vektör hafızasına dönüştürdünüz.

Öğrendikleriniz

1. **Ham metinden yapay zeka hafızasına**: Cognee'nin yapılandırılmamış verileri nasıl alıp, birleşik bir vektör + bilgi grafiği mimarisi kullanarak akıllı, aranabilir bir hafızaya dönüştürdüğünü öğrendiniz.

2. **Memify ile grafik zenginleştirme**: Temel grafik oluşturmanın ötesine geçerek, mevcut grafiğinizin üzerine türetilmiş bilgiler ve daha zengin ilişkiler eklemek için memify'ı nasıl kullanacağınızı öğrendiniz.

3. **Çoklu arama stratejileri**: Ajanınızın ihtiyaçlarına bağlı olarak farklı arama türleriyle (grafik farkındalıklı Soru-Cevap, RAG tarzı tamamlama, içgörüler, ham parçalar, kod arama vb.) hafızayı nasıl sorgulayacağınızı öğrendiniz.

4. **Görsel keşif**: Cognee'nin oluşturduğu bilgiyi grafik görselleştirmeleri ve Cognee UI kullanarak nasıl inceleyip hata ayıklayabileceğinizi, böylece bilginin nasıl yapılandırıldığını görebileceğinizi öğrendiniz.

5. **Oturum farkındalıklı hafıza**: Ajanların kullanıcılar arasında bilgi sızdırmadan oturumlar arası hatırlama yapabilmesi için oturum bazlı bağlamı kalıcı semantik hafıza ile nasıl birleştireceğinizi öğrendiniz.


## Temel Çıkarımlar
1. Gömülerle Desteklenen Bir Bilgi Grafiği Olarak Bellek

    - **Yapılandırılmış Anlayış**: Cognee, verilerinizi hem anlamına göre aranabilir hem de ilişkilerle bağlantılı hale getiren bir vektör deposu ve grafik deposunu birleştirir. Cognee varsayılan olarak dosya tabanlı veritabanları kullanır (vektör için LanceDB, grafik veritabanı için Kuzu).

    - **İlişki farkındalığıyla erişim**: Yanıtlar yalnızca "benzer metin" temelinde değil, aynı zamanda varlıkların nasıl ilişkili olduğuna dayanarak da oluşturulabilir.

    - **Canlı bellek**: Bellek katmanı evrilir, büyür ve tek bir bağlı grafik olarak sorgulanabilir durumda kalır.

2. Arama ve Akıl Yürütme Modları
    - **Hibrit erişim**: Arama, vektör benzerliği, grafik yapısı ve LLM akıl yürütmesini birleştirir; ham parça aramadan grafik farkındalıklı soru yanıtlamaya kadar.

    - **Göreve uygun mod seçimi**: Doğal dil yanıtları istediğinizde tamamlama tarzı modları, ajanınızın ham bağlama ihtiyaç duyduğu veya kendi akıl yürütmesini yönlendirdiği durumlarda parça/özet/grafik modlarını kullanın.

3. Kişiselleştirilmiş, Oturum Farkındalıklı Ajanlar
    - **Oturum bağlamı + uzun vadeli bellek**: Cognee, kısa vadeli "konu" bağlamını uzun vadeli, kullanıcı veya organizasyon düzeyindeki bellekten ayrı tutar.

## Gerçek Dünya Uygulamaları

1. **Dikey AI Ajanları**

    Bu not defterindeki modeli kullanarak, Cognee'yi erişim ve akıl yürütme çekirdeği olarak kullanan alan bilgisine sahip yardımcılar oluşturabilirsiniz:

- **Geliştirici yardımcıları**: Kod inceleme, olay analizi ve mimari asistanları, kod, API'ler, tasarım belgeleri ve biletleri tek bir bellek grafiği olarak tarar.

- **Müşteri odaklı yardımcılar**: Ürün belgeleri, SSS'ler, CRM notları ve geçmiş biletlerden grafik farkındalıklı erişim ve kaynaklı yanıtlar çeken destek veya başarı ajanları.

- **Dahili uzman yardımcılar**: Politika, hukuk veya güvenlik asistanları, izole edilmiş PDF'ler yerine birbirine bağlı kurallar, yönergeler ve geçmiş kararlar üzerinde akıl yürütür.

    Cognee, AI ajanları için kalıcı, doğru bir bellek olarak açıkça konumlandırılmıştır ve ajanınızın arkasına yerleşerek vektör depoları ve özel grafik kodlarının rastgele kombinasyonlarının yerini alır.

2. **Veri Silolarını Tek Bir Bellekte Birleştirme**

    Aynı yaklaşım, dağınık kaynaklar arasında birleşik bir bellek katmanı oluşturmanıza da yardımcı olur:

- **Silolardan tek bir grafiğe**: Yapılandırılmış (ör. veritabanları) ve yapılandırılmamış verileri (ör. belgeler, sohbetler) ayrı sistemler için ayrı dizinler yerine gömülerle desteklenen tek bir grafiğe aktarın.

- **Kaynaklar arası akıl yürütme ve atıflar**: Her şey üzerinde çok adımlı akıl yürütme yapın—grafik aracılığıyla günlükleri, metrikleri ve belgeleri "birleştirin"—ve yine de kaynaklı yanıtlar döndürün.

- **Bilgi merkezleri**: Bankacılık veya eğitim gibi alanlar için Cognee, PDF'leri, dahili sistemleri ve uygulama verilerini bir bilgi grafiğinde birleştirmek ve ajanların kesin, kaynaklı bağlamla soruları yanıtlamasını sağlamak için zaten kullanılmaktadır.

## Sonraki Adımlar

Temel bellek döngüsünü uyguladınız. İşte kendi başınıza deneyebileceğiniz doğal genişletmeler (ayrıntılar için [Cognee dokümantasyonuna](https://docs.cognee.ai/) bakın):

1. **Zamansal farkındalıkla deney yapın**: Metinden olayları ve zaman damgalarını çıkarmak için zamansal cogniyayı etkinleştirin.

2. **Ontoloji tabanlı akıl yürütme ekleyin**: Alanınız için bir OWL ontolojisi tanımlayın. Çıkarılan varlıklar ve ilişkiler bu şemaya dayandırılarak grafik kalitesini ve alan özelindeki yanıtları iyileştirmek için Cognee’nin ontoloji desteğini kullanın.

3. **Bir geri bildirim döngüsü ekleyin**: Gerçek kullanıcı geri bildirimlerinden grafik kenar ağırlıklarını ayarlaması için Cognee'ye izin verin, böylece erişim zamanla iyileşir ve statik kalmaz.

4. **Kişiselleştirme ve oturum davranışı için ayarlayın**: Her kişi veya ekibe paylaşılan bellek motoru üzerinde kendi görünümünü vermek için kullanıcı kimlikleri, kiracılar ve veri setlerini kullanın.

5. **Daha karmaşık ajanlara ölçeklendirin**: Cognee'yi ajan çerçevelerine bağlayarak aynı bellek katmanını paylaşan çoklu ajan sistemleri oluşturun. *Microsoft Agent Framework x Cognee eklentisi yakında geliyor.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul edilmez.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
